In [2]:
from src.guess.guess import ConfigType, Guess, Config
from src.domain.datasets.UnixCommandDataset import UnixCommandDataset  # Register dataset
from src.domain.datasets.ProjectEulerDataset import ProjectEulerDataset

config = Config(ConfigType.BART_RISC2ARM.get_path())
guess = Guess(config=config)
predictions = guess.guess()

Loading BART model: celinelee/bartlarge_risctoarm_cloze2048


/opt/homebrew/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2BD1B165-EC09-3F68-BCE4-8FE4E70CA7E2> /opt/homebrew/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <2A8DB508-8AAF-3FF1-BDFE-9EF17CC2B482> /opt/homebrew/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-10 02:08:23,287 - BartLargeModel - INFO - Initialized BartLargeModel on device: mps
2025-04-10 02:08:23,291 - src.helpers.dataset - INFO - Loaded 11 instances
2025-04-10 02:08:23,291 - src.helpers.data_loader - INFO - Loaded 

Source JSONL: data/processed/RISCV/UnixCommands_risc.jsonl
Target JSONL: data/processed/ARM64/UnixCommands_arm.jsonl
Loading file: data/processed/RISCV/UnixCommands_risc.jsonl
Loaded 11 entries from data/processed/RISCV/UnixCommands_risc.jsonl
Loading file: data/processed/ARM64/UnixCommands_arm.jsonl
Loaded 11 entries from data/processed/ARM64/UnixCommands_arm.jsonl
Source entries: 11
Target entries: 11
Creating instance 0 with source key risc and target key arm
Source entry keys: dict_keys(['source', 'risc', 'risc_output', 'risc_verbose'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output', 'arm_verbose'])
Creating instance 1 with source key risc and target key arm
Source entry keys: dict_keys(['source', 'risc', 'risc_output', 'risc_verbose'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output', 'arm_verbose'])
Creating instance 2 with source key risc and target key arm
Source entry keys: dict_keys(['source', 'risc', 'risc_output', 'risc_verbose'])
Target entry keys: 

BartModel is using BartSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Token indices sequence length is longer than the specified maximum sequence length for this model (2189 > 2048). Running this sequence through the model will result in indexing errors


In [4]:
%load_ext autoreload
%autoreload 2

In [109]:
from src.sketch.sketch import Sketch

sketch = Sketch(config, guess.model)

In [85]:
import pandas as pd
from IPython.display import HTML, display

def html_escape(text):
    # Convert special characters to HTML-safe versions
    return (text.replace("&", "&amp;")
                .replace("<", "&lt;")
                .replace(">", "&gt;")
                .replace("\n", "<br>")
                .replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;"))  # 4 spaces for a tab

def display_blocks(blocks, pred):
    data = []

    for block in blocks:
        source_text = sketch.model.tokenizer.decode(
            pred.source[0][block.source_start:block.source_end]
        )
        pred_text = sketch.model.tokenizer.decode(
            pred.pred[0][block.pred_start:block.pred_end]
        )
        data.append({
            'SOURCE': html_escape(source_text),
            'PRED': html_escape(pred_text)
        })

    df = pd.DataFrame(data)

    styles = """
    <style>
    table {
        table-layout: auto;
        word-wrap: break-word;
    }
    td {
        white-space: normal !important;
        font-family: monospace;
        vertical-align: top;
    }
    </style>
    """
    display(HTML(styles + df.to_html(escape=False)))


In [110]:
for instance_id, pred in predictions.items():
    print(instance_id)
    line_mappings = sketch.map_predicted_lines_to_source_lines(
        pred.source,
        pred.pred,
        pred.alignments
    )

    blocks = sketch.extract_pure_instruction_blocks(pred.source, pred.pred, line_mappings)
    display_blocks(blocks, pred)

UnixCommands/cat


,SOURCE,PRED
0,"mv a5,a0","mov x29, sp"
1,"li a4, 0","mov x1, 0"
2,"addi a5,a5,8","add x0, x0, 8"
3,"addi a5,s0,-128 mv a2,a4 mv a1,a5","add x0, sp, 48 mov x2, x1 mov x1, x0"
4,"addi a4,s0,-128","add x0, sp, 48"
5,"li a2,99 mv a1,a4","mov x2, 99 mov x1, x0"
6,"xor a5, a3, a5 li a3, 0","subs x3, x3, x2 mov x2, 0"


UnixCommands/cd


,SOURCE,PRED
0,"addi sp,sp,-1072","sub sp, sp, #1072"
1,"mv a5,a0","mov x29, sp"
2,"li a4, 0","mov x1, 0"
3,"addi a5,s0,-1048 li a1,1024","add x0, sp, 40 mov x1, 1024"
4,"li a0,1","cmp x0, 0"
5,"addi s0,sp,1072 mv a5,a0","add x0, sp, 40 mov x2, x0"
6,"mv a0,a5","mov x0, x3"
7,"xor a5, a3, a5 li a3, 0","subs x3, x3, x2 mov x2, 0"
8,"addi sp,sp,-1072","add sp, sp, 1072"


UnixCommands/cp


...

,SOURCE,PRED
0,"mv a5,a0","mov x29, sp"
1,"li a4, 0","mov x1, 0"
2,"addi a5,a5,8","add x0, x0, 8"
3,"mv a0,a5","mov x0, x2"
4,"addi a5,s0,-128 mv a1,a5","add x0, sp, 48 mov x1, x0"
5,"addi a5,s0,-128 mv a1,a5","add x0, sp, 48 mov x1, x0"
6,"addi a5,s0,-128 mv a2,a5","add x0, sp, 48 mov x2, x0"


UnixCommands/touch


,SOURCE,PRED
0,"mv a5,a0","mov x29, sp"
1,"addi a5,a5,8","add x0, x0, 8"
2,"addi a5,a5,8","add x0, x0, 8"
3,"li a1,0","mov x1, 0"


UnixCommands/xargs


,SOURCE,PRED
0,"li a4, 0","mov x29, sp"
1,"li a4, 0","mov x1, 0"
2,"addi a5,s0,-264 mv a2,a5","add x0, sp, 40 mov x2, x0"
3,"addi a5,s0,-264 mv a2,a4","add x0, sp, 40 mov x2, x1"
4,"li a1,100","cmp x0, 0"
